In [1]:
from csv_parser import csvRDD

hsfn = 'DOE_High_School_Directory_2014-2015.csv'
satfn = 'SAT_Results.csv'

lines_hs = sc.textFile(hsfn, use_unicode=False).cache()
lines_sat = sc.textFile(satfn, use_unicode=False).cache()

rdd_hs = csvRDD(lines_hs)
rdd_sat = csvRDD(lines_sat)

# from hs, filter out lines that contain total_student <500 or 'N/A', return (dbn, boro)
records_hs = rdd_hs.filter(lambda line: line[17]!='N/A') \
                   .filter(lambda line: int(line[17])>=500) \
                   .map(lambda line: (line[0], line[2]))

# from sat, filter out lines that contain 's' in Math Avg, return (dbn, Math Avg, # takers)
records_sat = rdd_sat.filter(lambda line: line[4]!='s') \
                     .map(lambda line: (line[0], line[4], line[2]))

In [2]:
hsdf = sqlContext.createDataFrame(records_hs, ("dbn","boro"))

satdf = sqlContext.createDataFrame(records_sat, ("dbn","math_avg","total_takers"))

joindf = hsdf.join(satdf, hsdf.dbn == satdf.dbn, 'inner').drop(satdf.dbn)
joindf.registerTempTable('school_sat')
joindf.show()

+------+---------+--------+------------+
|   dbn|     boro|math_avg|total_takers|
+------+---------+--------+------------+
|15K519| Brooklyn|     402|          77|
|10X440|    Bronx|     426|         443|
|14K558| Brooklyn|     385|          72|
|07X527|    Bronx|     378|          62|
|27Q262|   Queens|     419|          74|
|30Q501|   Queens|     496|         174|
|30Q502|   Queens|     434|         148|
|10X445|    Bronx|     688|         731|
|04M435|Manhattan|     514|         375|
|16K498| Brooklyn|     372|          46|
|01M539|Manhattan|     574|         159|
|14K449| Brooklyn|     584|          72|
|03M479|Manhattan|     575|         261|
|12X271|    Bronx|     378|          46|
|27Q650|   Queens|     491|         194|
|09X241|    Bronx|     464|          80|
|10X225|    Bronx|     391|          59|
|29Q259|   Queens|     397|          46|
|25Q281|   Queens|     463|          59|
|03M415|Manhattan|     368|          32|
+------+---------+--------+------------+
only showing top

In [3]:
sqlContext.sql('select boro, (sum(math_avg*total_takers)/sum(total_takers)) as avg \
from school_sat group by boro').collect()

[Row(boro=u'Queens', avg=474.88703080977916),
 Row(boro=u'Brooklyn', avg=487.46256168204246),
 Row(boro=u'Staten Island', avg=477.9099864130435),
 Row(boro=u'Bronx', avg=470.198606271777),
 Row(boro=u'Manhattan', avg=514.9312780989081)]